# Autoencoder

## Unfortunately , it doesn't work very well, and I guess it is because features it captures are too simple, and RNN or similiar method should be the only way to generate text.

In [ ]:
embedding_size = 100
batch_size = 8

In [ ]:
embedding_matrix_2 = load_glove(word_index)

In [ ]:
class encoder(nn.Module):
    def sample(self,mu,logvar): # reparametrize
        eps = Variable(torch.randn(mu.size()))# generate the standard guass rv having the same size with mu
        eps = eps.cuda()
        std = torch.exp(logvar/2.0)
        return mu+ eps * std   # Using reparametrization for backprop
        

In [ ]:
class encoderrnn(encoder):
    def __init__(self, input_size=embedding_size, hidden_size=70, output_size=latent, n_layers=2, bidirectional=True):
        super(encoderrnn, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.bidirectional = bidirectional
        self.embedding = nn.Embedding(max_word, embedding_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix_2, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.lstm = nn.LSTM(input_size , hidden_size, num_layers=n_layers,
                            bidirectional=bidirectional, batch_first=True,dropout=0.1)
        self.linearout = nn.Linear(hidden_size*2*maxq, output_size*2)
    def forward(self, input):
        h_embedding = self.embedding(input)
        h_lstm, _ = self.lstm(h_embedding)
        out = h_lstm.contiguous().view(-1,self.hidden_size*2*maxq)
        out = self.linearout(out)
        mu,logvar = torch.chunk(out,2,dim=1)
        z = torch.tensor(self.sample(mu, logvar))
        return mu,logvar,z,h_embedding

In [ ]:
class decoderrnn(nn.Module):
    def __init__(self, input_size=latent, hidden_size=70, 
                 output_size=embedding_size, nlayers=3, dropout=0.1):
        super(decoderrnn,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.nlayers = nlayers
        self.dropout = dropout
        self.linear1 = nn.Linear(input_size, self.hidden_size*maxq)
        self.lstm = nn.LSTM(hidden_size,output_size, num_layers= nlayers, bidirectional=True)

        
        
    def forward(self, z):
        out = self.linear1(z)
        out = out.contiguous().view(-1,maxq,self.hidden_size)
        out,_ = self.lstm(out)

        out = out[:,:,:self.output_size]+out[:,:,self.output_size:]
        return out
    
    def generate(self,rv):
        with torch.no_grad():
            embed = self.forward(rv)
        return embed

In [ ]:
class vae(nn.Module):
    def __init__(self, encoder, decoder):
        super(vae,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    def forward(self,inputs):
        mu, v, z,embed = self.encoder(inputs)
        gen = self.decoder(z)
        return mu, v, gen,embed

In [ ]:
def compute_KL(mu,v):
    out = torch.sum(torch.exp(2*v),1)+ torch.sum(torch.pow(mu,2),1)-torch.log(torch.sum(torch.exp(2*v),1))
    return out

In [ ]:
en = encoderrnn()
de = decoderrnn()
model2 = vae(en,de)
model2.to(device)

In [ ]:
optimizer = torch.optim.Adam(model2.parameters(),lr=0.5)

In [ ]:
trainfoldx = torch.tensor(train_X[train_y==1], dtype=torch.long).to(device)
train = torch.utils.data.TensorDataset(trainfoldx)
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size,shuffle=True)

In [ ]:
for epoch in range(n_epochs):
    start = time.time()
    grand_loss = 0
    for  batch_idx, (x_batch) in enumerate(train_loader):
            #forward
        mu, v, gen,embedding = model2(x_batch[0])
        KL = compute_KL(mu,v)
        KL = KL.mean()
        Fit =torch.max(torch.abs(gen-embedding),2)[0].mean()*150
        loss = Fit - KL
            
            #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Loss: %.4f' 
                   %(epoch+1, n_epochs, batch_idx, len(train_loader), loss.item()))